In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, HTML
from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import threading

In [2]:
icons = ["️️👧🏻", "️️🦩", "️️🦢", "️️🦘", "🏠"]
friends = []
helperIcons = {"A":"🚒", "B":"🛶"}
helpers = []
obstacleIcons = {"Z":"🔥", "Y":"🌊"}
obstacleToHelperMap = {"🔥":"🚒", "🌊":"🛶"}
player = None
df = None
maze = None
level = 1

def backgroundColor(cell):
    color = 'moccasin' if cell == "️️👧🏻" else 'white'
    return 'background-color: %s' % color

def initMaze(n):
    global df, maze, player
    df = pd.read_csv("./mazes/"+str(n)+".txt", header=None)
    m = df.to_numpy()
    m[m == "#"] = "️️🔲"
    for i in range(len(icons)):
        m[m == str(i)] = icons[i]
    for k in helperIcons:
        m[m == k] = helperIcons[k]
    for k in obstacleIcons:
        m[m == k] = obstacleIcons[k]
        
    l = np.where(m == icons[0])
    player = ({'icon':icons[0], 'location':(l[0][0], l[1][0])})
    
    maze = df.style.set_properties(**{'height': '50px', 'width':'50px', 'font-size':'50px', 'padding':'0px', 
                                      'margin':'0px', 'line-height':'1'}).applymap(backgroundColor)



In [3]:
def getCurrentStatus():
    status = "<span style='font-size: 40px;'>Friends: " + " ".join(friends) + "</span>"
    if len(helpers) > 0 :
        status += "<br/><span style='font-size: 40px;'>Helpers: " + " ".join(helpers) + "</span>"
    return status

In [4]:
output = widgets.Output()
board = widgets.HBox([output], layout=Layout())
restartButton = widgets.Button(description='Restart',button_style='danger')
levelMenu = widgets.Dropdown(
       options=list(range(1,5)),
       value=1,
       description='Level: ')
status = None

def refreshMaze():
    output.clear_output(wait=True)
    with output:
        display(maze)
        
def displayMaze():
    global status
    status = HTML(getCurrentStatus())
    display(restartButton, levelMenu, board, status)
    refreshMaze()

In [5]:
autoMoveSpeed = 0.5
autoMoveTimer = None
def autoMove(key):
    global autoMoveTimer
    autoMoveTimer = threading.Timer(autoMoveSpeed, move, [key])
    autoMoveTimer.start()

In [6]:
def move(key):
    global chars, status, friends, helpers
    if autoMoveTimer:
        autoMoveTimer.cancel()
    nl = None
    if key == "ArrowDown":
        nl = (player['location'][0]+1, player['location'][1])
    elif key == "ArrowLeft":
        nl = (player['location'][0], player['location'][1]-1)
    elif key == "ArrowRight":
        nl = (player['location'][0], player['location'][1]+1)
    elif key == "ArrowUp":
        nl = (player['location'][0]-1, player['location'][1])
    
    if nl[0] >= 0 and nl[0] < df.shape[0] and nl[1] >= 0 and nl[1] < df.shape[1]:
        if df.iloc[nl] == '️️🔲' or df.iloc[nl] == '🏠':
            if autoMoveTimer:
                autoMoveTimer.cancel()
            if df.iloc[nl] == '🏠' and len(friends) == len(icons)-2:
                status.value = "<span style='color:green; font-size: 40px;'>You Are Home!</span>"
            return
        updateStatus = False
        if df.iloc[nl] in icons and not df.iloc[nl] == icons[-1]:
            # Collect a friend
            if not df.iloc[nl] in friends:
                friends.append(df.iloc[nl])
                updateStatus = True
        elif df.iloc[nl] in helperIcons.values():
            # Collect a helper
            if not df.iloc[nl] in helpers:
                helpers.append(df.iloc[nl])
                updateStatus = True
        elif df.iloc[nl] in obstacleIcons.values():
            # Encounter an obstacles
            h = obstacleToHelperMap[df.iloc[nl]]
            if not h in helpers:
                if autoMoveTimer:
                    autoMoveTimer.cancel()
                status.value = "<span style='color:red; font-size: 40px;'>Game Over!</span>"
                boardEvent.on_dom_event(None)
                return
        df.iloc[player['location'] ] = " "
        df.iloc[nl] = player['icon']
        player['location'] = nl
        refreshMaze()
        status.value = getCurrentStatus()
        autoMove(key)

In [7]:
# Listen to keyboard events
boardEvent = Event(source=output, watched_events=['keydown'])
def handleEvent(event):
    if event['code'] in ['ArrowDown', 'ArrowUp', 'ArrowLeft', 'ArrowRight']:
        move(event['code'])

In [8]:
def restartGame(_):
    global friends, level, helpers
    if autoMoveTimer:
        autoMoveTimer.cancel()
    level = levelMenu.value
    initMaze(level)
    friends = []
    helpers = []
    status.value = getCurrentStatus()
    refreshMaze()
    boardEvent.on_dom_event(handleEvent)
restartButton.on_click(restartGame)

In [9]:
def startGame():
    initMaze(level)
    displayMaze()
    boardEvent.on_dom_event(handleEvent)

In [10]:
startGame()

Button(button_style='danger', description='Restart', style=ButtonStyle())

Dropdown(description='Level: ', options=(1, 2, 3, 4), value=1)

HTML(value="<span style='font-size: 40px;'>Friends: </span>")